In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
import os 
import tensorflow as tf
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # Mke model run on CPU, the GPU version is unstable 
print(tf.__version__)
import os
import re
import shutil
import tensorflow as tf

2.0.0-alpha0


Task 6 development:


*LOAD PROCESSED IMAGES*

* Load training set 

* Load test set


*Model Creation*

* Models :


* 2 Hidden Layer NN + Batch normalization
* 2 Hidden Layer NN + LASSO REGULARIZED MODEL
* Commmite Neural networks 
 



*Model evaluation*

* Accuracy over Epochs 
* Loss over Epochs 

In [17]:
# setting seeeds 
import numpy as np
np.random.seed(1)
import tensorflow as tf

tf.random.set_seed(1)

# Load processed data  for Modelling 


In [18]:
import pickle

In [19]:
training_set_features = pickle.load(open( "Dataset/processed/training_features.pickle", "rb" ) )

training_set_labels = pickle.load(open( "Dataset/processed/training_labels.pickle", "rb" ) )

In [20]:
len(training_set_labels)

20198

In [21]:
testing_set_features = pickle.load(open( "Dataset/processed/test_features.pickle", "rb" ) )

testing_set_labels = pickle.load(open( "Dataset/processed/test_labels.pickle", "rb" ) )

In [22]:
len(testing_set_labels)

4942

# Create the TF.Dataset testing and training sets


In [23]:
tf_test_dataset=tf.data.Dataset.from_tensor_slices((testing_set_features,testing_set_labels)) # pass in data as tuple
tf_test_dataset

<TensorSliceDataset shapes: ((2304,), ()), types: (tf.float64, tf.float64)>

In [24]:
tf_training_dataset=tf.data.Dataset.from_tensor_slices((training_set_features,training_set_labels)) # pass in data as tuple
tf_training_dataset

<TensorSliceDataset shapes: ((2304,), ()), types: (tf.float32, tf.int8)>

# Advanced models 

* Batch Norm model (3-HIDDDEN LAYERS ) 

* LASSO MODEL (3 - HIDDEN LAYERS )  


## Setting the training parameters

In [26]:
# Network training parameters
EPOCHS = 100 #number of training epochs
VERBOSE = 1
NB_CLASSES = 4 # number of outputs = number of emotions
N_HIDDEN = 200
initializer = tf.keras.initializers.GlorotNormal(seed=1)

## Batch Normalization model

In [27]:
from tensorflow.keras.callbacks import TensorBoard

In [28]:
batch_norm_model = tf.keras.models.Sequential( name = 'Batch Norm Model' )

batch_norm_model.add(keras.layers.Dense (N_HIDDEN,input_shape = (2304,), name='dense_layer_1', 
                                         kernel_initializer = initializer,
                                         activation='relu'))

batch_norm_model.add(keras.layers.BatchNormalization()) #batch norm layer 

batch_norm_model.add(keras.layers.Dense (N_HIDDEN, name='dense_layer_2', 
                                         kernel_initializer = initializer,
                                         activation='relu'))

batch_norm_model.add(keras.layers.BatchNormalization()) # batch norm layer

batch_norm_model.add(keras.layers.Dense( NB_CLASSES, name='output_layer', kernel_initializer = initializer, 
                                  activation = 'softmax'  ) )

In [29]:
print(batch_norm_model.summary())


Model: "Batch Norm Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer_1 (Dense)        (None, 200)               461000    
_________________________________________________________________
batch_normalization_v2_2 (Ba (None, 200)               800       
_________________________________________________________________
dense_layer_2 (Dense)        (None, 200)               40200     
_________________________________________________________________
batch_normalization_v2_3 (Ba (None, 200)               800       
_________________________________________________________________
output_layer (Dense)         (None, 4)                 804       
Total params: 503,604
Trainable params: 502,804
Non-trainable params: 800
_________________________________________________________________
None


In [30]:
# Save architecture

#tf.keras.utils.plot_model(batch_norm_model,to_file='model architectures/batch_norm.png')

In [31]:
batch_norm_model.compile(optimizer = 'ADAM', 
              loss = 'sparse_categorical_crossentropy',  # If LABELS---> Integers, then sparse is used. IF one-hot, then CE is used
              metrics=['accuracy'])

In [32]:
#shuffle data
shuffled_set=tf_training_dataset.shuffle(50_000)#.batch(64)    # This represents X_train and Y_train 

shuffled_set

<ShuffleDataset shapes: ((2304,), ()), types: (tf.float32, tf.int8)>

In [33]:
#batched_training_set = shuffled_set.batch(64) #batch the dataset into 64,128, etc... batches

In [35]:
from datetime import datetime
now = datetime.now()

In [36]:
#Now we want to make our TensorBoard callback object:
tf.random.set_seed(1)

NAME = "Batch Norm Model\\"
now = datetime.now()

logdir = "tf_logs\\"+ NAME + now.strftime("%Y%m%d-%H%M%S") + "\\" 

tensorboard = TensorBoard(logdir)
batch_norm_model.fit(shuffled_set.batch(64),shuffle=False, epochs=EPOCHS, verbose=VERBOSE, callbacks=[tensorboard])

Epoch 1/100
316/316 [==============================] - 3s 10ms/step - loss: 1.3027 - accuracy: 0.4207
Epoch 2/100
316/316 [==============================] - 2s 6ms/step - loss: 1.2158 - accuracy: 0.4633
Epoch 3/100
316/316 [==============================] - 2s 6ms/step - loss: 1.1791 - accuracy: 0.4827
Epoch 4/100
316/316 [==============================] - 2s 6ms/step - loss: 1.1506 - accuracy: 0.4990
Epoch 5/100
316/316 [==============================] - 2s 6ms/step - loss: 1.1220 - accuracy: 0.5178
Epoch 6/100
316/316 [==============================] - 2s 6ms/step - loss: 1.1072 - accuracy: 0.5229
Epoch 7/100
316/316 [==============================] - 2s 6ms/step - loss: 1.0912 - accuracy: 0.5356
Epoch 8/100
316/316 [==============================] - 2s 6ms/step - loss: 1.0756 - accuracy: 0.5428
Epoch 9/100
316/316 [==============================] - 2s 6ms/step - loss: 1.0553 - accuracy: 0.5531
Epoch 10/100
316/316 [==============================] - 2s 6ms/step - loss: 1.0225 - accur

In [37]:
#evalute the model
test_loss, test_acc = batch_norm_model.evaluate(tf_test_dataset.batch(64))
print('Test accuracy:', test_acc)

78/78 [==============================] - 0s 5ms/step - loss: 8.0754 - accuracy: 0.3153
Test accuracy: 0.31525698


In [38]:
# making prediction
batch_norm_predictions = batch_norm_model.predict(tf_test_dataset.batch(32))
batch_norm_predictions

array([[1.1819166e-07, 4.2159173e-01, 4.8770826e-02, 5.2963728e-01],
       [5.0300814e-02, 7.8212280e-12, 9.4806862e-01, 1.6304669e-03],
       [4.1495891e-08, 1.0042367e-12, 1.2807253e-01, 8.7192744e-01],
       ...,
       [1.1617627e-08, 1.2579955e-10, 9.7261488e-01, 2.7385151e-02],
       [7.7832124e-07, 1.7530478e-04, 2.9346445e-09, 9.9982399e-01],
       [9.9753475e-01, 1.6431301e-07, 2.4651224e-03, 5.1496905e-08]],
      dtype=float32)

In [39]:
# save model
batch_norm_model.save('trained models/my_batch_model') 

## LASSO Regularized Model

In [40]:
LASSO_regularizer = tf.keras.regularizers.l1_l2( l1 = 0.0001, l2 = 0.0002)

In [41]:
lasso_model = tf.keras.models.Sequential( name = 'LASSO MODEL' )

lasso_model.add(keras.layers.Dense (N_HIDDEN,input_shape = (2304,), name='dense_layer_1', 
                                         kernel_initializer = initializer,
                                         kernel_regularizer = LASSO_regularizer,
                                         activation='relu'))

lasso_model.add(keras.layers.Dense (N_HIDDEN, name='dense_layer_2', 
                                         kernel_initializer = initializer,
                                         kernel_regularizer = LASSO_regularizer,
                                         activation='relu'))

lasso_model.add(keras.layers.Dense( NB_CLASSES, name='output_layer', kernel_initializer = initializer, 
                                  activation = 'softmax'  ) )

In [42]:
print(lasso_model.summary())

Model: "LASSO MODEL"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer_1 (Dense)        (None, 200)               461000    
_________________________________________________________________
dense_layer_2 (Dense)        (None, 200)               40200     
_________________________________________________________________
output_layer (Dense)         (None, 4)                 804       
Total params: 502,004
Trainable params: 502,004
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
lasso_model.compile(optimizer = 'ADAM', 
                    loss = 'sparse_categorical_crossentropy',  # If LABELS---> Integers, then sparse is used. IF one-hot, then CE is used
                    metrics = ['accuracy'])

In [46]:
from datetime import datetime

In [47]:
#Now we want to make our TensorBoard callback object:
NAME = "Lasso Model\\"

now = datetime.now()

logdir = "tf_logs\\"+ NAME + now.strftime("%Y%m%d-%H%M%S") + "\\" 

tensorboard = TensorBoard(logdir)

In [48]:
tf.random.set_seed(1)

lasso_model.fit(shuffled_set.batch(64), shuffle=False, epochs=EPOCHS, verbose=VERBOSE, callbacks=[tensorboard])

Epoch 1/100
316/316 [==============================] - 2s 6ms/step - loss: 2.0092 - accuracy: 0.3734
Epoch 2/100
316/316 [==============================] - 1s 4ms/step - loss: 1.5069 - accuracy: 0.4130
Epoch 3/100
316/316 [==============================] - 1s 4ms/step - loss: 1.3884 - accuracy: 0.4269
Epoch 4/100
316/316 [==============================] - 1s 4ms/step - loss: 1.3426 - accuracy: 0.4316
Epoch 5/100
316/316 [==============================] - 1s 4ms/step - loss: 1.3180 - accuracy: 0.4411
Epoch 6/100
316/316 [==============================] - 1s 4ms/step - loss: 1.3009 - accuracy: 0.4450
Epoch 7/100
316/316 [==============================] - 1s 4ms/step - loss: 1.2820 - accuracy: 0.4513
Epoch 8/100
316/316 [==============================] - 1s 4ms/step - loss: 1.2775 - accuracy: 0.4559
Epoch 9/100
316/316 [==============================] - 1s 4ms/step - loss: 1.2668 - accuracy: 0.4588
Epoch 10/100
316/316 [==============================] - 1s 4ms/step - loss: 1.2636 - accura

In [49]:
#evalute the model
test_loss, test_acc = lasso_model.evaluate(tf_test_dataset.batch(64))
print('Test accuracy:', test_acc)

78/78 [==============================] - 0s 4ms/step - loss: 1.2766 - accuracy: 0.4686
Test accuracy: 0.46863618


In [50]:
# making prediction
lasso_predictions = lasso_model.predict(tf_test_dataset.batch(64))
lasso_predictions

array([[0.24362206, 0.2334331 , 0.17738259, 0.3455623 ],
       [0.27760324, 0.10691798, 0.46112278, 0.154356  ],
       [0.15562165, 0.49117714, 0.20603107, 0.14717007],
       ...,
       [0.27959532, 0.02071059, 0.2683908 , 0.43130326],
       [0.17038801, 0.2510769 , 0.30984694, 0.26868817],
       [0.2995861 , 0.57075614, 0.01312324, 0.11653463]], dtype=float32)

In [51]:
# save model
lasso_model.save('trained models/my_lasso_model') 

# END OF NOTEBOOK 

### RESULTS CAN BE SEEN WITH TENSORBOARD and in the Report 